# Train ANN with DSM2 data

This notebook uses a memory input with CNN, GRU and desnse layer(s) to get a fit for the DSM2 produced data. Again the aim is to show the effectiveness of all of the above layers as potential ANN model layers that show decent performance

In [ ]:
import numpy as np
import pandas as pd

import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import layers
#import keras

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

import hvplot.pandas
import panel as pn
import holoviews as hv
hv.extension('bokeh')

import os

In [ ]:
print(tf.__version__, np.__version__, pd.__version__, pn.__version__, hv.__version__)

In [ ]:
import annutils

In [ ]:
dflist = [pd.read_excel('./dsm2_ann_BaselineData_20220120.xlsx',i,index_col=0,parse_dates=True) for i in range(8)]

In [ ]:
dfinps = pd.concat(dflist[0:7],axis=1)
dfinps.head()

In [ ]:
dfouts = dflist[7]

In [ ]:
dfouts

## Tensorflow Board Setup
A log directory to keep the training logs

Tensorboard starts a separate process and is best started from the command line. Open a command window and activate this environment (i.e. keras) and goto the current directory. Then type in
```
tensorboard --logdir=./tf_training_logs/ --port=6006
```

In [ ]:
from tensorflow import keras

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir=./tf_training_logs/ --port=6006

In [ ]:
root_logdir = os.path.join(os.curdir, "tf_training_logs")

# Calibration and Validation Periods
Calibration is from 1940 - 2015 and Validation from 1923 - 1939 as per the Calsim 3 ANN paper

The output locations are names of the columns in the output(labels) csv files. For each location, an ANN is trained on all the specified data sets

In [ ]:
output_locations = list(dfouts.columns)
calib_slice = slice('1990', '2020')
valid_slice = slice('1990', '2000')

In [ ]:
dfinps, dfouts = annutils.synchronize(dfinps, dfouts)

In [ ]:
xs,ys = annutils.create_xyscaler(dfinps, dfouts.iloc[:,[0]])

In [ ]:
xtrain = xs.transform(dfinps) # time x ninputs

In [ ]:
ytrain = ys.transform(dfouts.iloc[:,[0]])

In [ ]:
Xt,Yt=annutils.create_memory_sequence_set(xtrain, ytrain, time_memory=120)

xx and yy are input and target time series synchronized on the same time 


This is has been transformed to a Xt and Yt
 * Xt is an array of number of batches x time memory x number of features
 * Yt is an array of the output 

In [ ]:
Xt.shape,Yt.shape

In [ ]:
tf.keras.backend.set_floatx('float64')

In [ ]:
# Define Sequential model with 3 layers
NFEATURES = 7  # 126  # (8 + 10)*7


def build_dense_model():
    model = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=[120, NFEATURES]),
        keras.layers.Flatten(),
        keras.layers.Dense(4, activation='sigmoid'),
        keras.layers.Dense(2, activation='sigmoid'),
        keras.layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=0.001), loss="mse")
    return model


def build_layer_from_string_def(s='i120'):
    if s[0:3] == 'c1d':
        fields = s[3:].split('x')
        return keras.layers.Conv1D(filters=int(fields[0]), kernel_size=int(fields[1]), strides=int(fields[2]),
                                   padding='causal', activation='linear')
    elif s[0:2] == 'td':
        return keras.layers.TimeDistributed(keras.layers.Dense(int(s[2:]), activation='elu'))
    elif s[0:2] == 'dr':
        return keras.layers.Dropout(float(s[2:]))
    elif s[0] == 'i':
        return keras.layers.InputLayer(input_shape=[int(s[1:]), NFEATURES])
    elif s[0] == 'f':
        return keras.layers.Flatten()
    elif s[0] == 'g':
        return keras.layers.GRU(int(s[1:]), return_sequences=True, activation='relu')
    elif s[0] == 'd':
        return keras.layers.Dense(int(s[1:]), activation='elu')
    elif s[0] == 'o':
        return keras.layers.Dense(int(s[1:]), activation='linear')
    else:
        raise Exception('Unknown layer def: %s' % s)


def build_model_from_string_def(strdef='i120_f_d4_d2_d1'):
    model = keras.models.Sequential(
        [build_layer_from_string_def(f) for f in strdef.split('_')])
    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=0.001), loss="mse")
    return model

In [ ]:
# Tensorflow Board Setup
#model_str_def = "i120_c1d5x1x1_c1d5x4x1_g4_g2_f_d4_d2_o1"
#model_str_def = "i120_f_d4_d2_o1"
#model_str_def = "i120_c1d15x1x1_c1d15x4x1_td4_td2_o1"
#model_str_def = "i120_g4_g2_f_o1"
model_str_def = 'i120_c1d10x3x1_f_d4_d2_o1'
tensorboard_cb = keras.callbacks.TensorBoard(os.path.join(root_logdir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
                                                          model_str_def))
# check if model has been run before
try:
    model = annutils.load_model(model_str_def).model
    #assuming that loaded model xs,ys is the same: FIXME:
    print('Using existing model from files for %s'%model_str_def)
except:
    print('Creating New Model: ')
    model = build_model_from_string_def(model_str_def)
model.summary()

In [ ]:
#
model.fit(Xt, Yt, epochs=1000, batch_size=120, 
#          validation_data=(Xt[split_point+365:],Yt[split_point+365:]),
          callbacks=[
              keras.callbacks.EarlyStopping(
                  monitor="loss", patience=25, mode="min", restore_best_weights=True),
              tensorboard_cb
          ],
          )

In [ ]:
annutils.save_model(model_str_def, model, xs, ys)

# Show the performance on the data sets visually

Change the location to one of the locations for which the ANN is trained and run cells below to see performance on one or more of the data sets

In [ ]:
Ytp = model.predict(Xt)[:,0].flatten()

dfp = pd.concat([pd.DataFrame(y,index=dfouts.index[120:]) for y in [Yt, Ytp]],axis=1)
dfp.columns=['actual','predicted']

dfp.hvplot(width=800,legend='top_right')

# Display weights and x and y scaling parameters


In [ ]:
annmodel = annutils.load_model(model_str_def)

annmodel.model.get_weights()
annmodel.xscaler.data_min_, annmodel.xscaler.data_max_
annmodel.xscaler.feature_range
annmodel.xscaler.min_
annmodel.xscaler.scale_

print('Ann model loaded for %s'%model_str_def)